In [1]:
# Standard imports
import pandas as pd
import numpy as np

In [2]:
# For plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
# For SQL queries
import sqlite3
conn = sqlite3.connect("/Users/Quicksilver/Desktop/Metis/sea19_ds10/Project03/FPA_FOD_20170508.sqlite")
cursor=conn.cursor()

# What are the column names?

In [4]:
query = '''
SELECT * FROM Fires LIMIT 5;
'''

columns_df = pd.read_sql_query(query, conn)
columns_df

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


In [5]:
columns_df.columns

Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
       'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME',
       'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME',
       'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'DISCOVERY_TIME', 'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE',
       'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY',
       'FIPS_CODE', 'FIPS_NAME', 'Shape'],
      dtype='object')

# Which columns are helpful?

In [6]:
columns_df_abbrev = columns_df[['FPA_ID', 
       'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR', 'CONT_DATE',
       'CONT_DOY', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'OWNER_CODE', 'OWNER_DESCR', 'STATE', 'COUNTY',
       'FIPS_CODE', 'FIPS_NAME',]].copy()
columns_df_abbrev

,FPA_ID,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,FS-1418826,2005,2453403.5,33,9.0,Miscellaneous,2453403.5,33,0.10,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas
1,FS-1418827,2004,2453137.5,133,1.0,Lightning,2453137.5,133,0.25,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer
2,FS-1418835,2004,2453156.5,152,5.0,Debris Burning,2453156.5,152,0.10,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado
3,FS-1418845,2004,2453184.5,180,1.0,Lightning,2453189.5,185,0.10,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine
4,FS-1418847,2004,2453184.5,180,1.0,Lightning,2453189.5,185,0.10,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine


## Columns to use in West Coast dataframe

In [7]:
columns_df_abbrev_west = columns_df[['FPA_ID', 
       'FIRE_YEAR', 'DISCOVERY_DOY',
       'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR',
       'CONT_DOY', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'STATE']].copy()
columns_df_abbrev_west.head(0)

,FPA_ID,FIRE_YEAR,DISCOVERY_DOY,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DOY,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE


# How many unique "FIRE_NAME" entries are there?

In [ ]:
query = '''
SELECT count(distinct FIRE_NAME)
FROM Fires;
'''
cursor.execute(query)
fire_names = cursor.fetchall()
fire_names

Answer: almost half a million. Not especially helpful.

# Create DataFrame for data from West Coast states: AK, WA, OR, CA

## How many states are there?

In [ ]:
query = '''
SELECT count(distinct STATE)
FROM Fires;
'''
cursor.execute(query)
states = cursor.fetchall()
states

Answer: 52 states (includes Puerto Rico and DC). Nothing strange here.

## How many fires happened in West Coast states?

In [ ]:
query = '''
SELECT STATE, count(distinct FPA_ID)
FROM Fires
WHERE (STATE='AK' OR STATE='WA' OR STATE='OR' OR STATE='CA') 
GROUP BY STATE;
'''
cursor.execute(query)
west_coast_fires = cursor.fetchall()
west_coast_fires

### Are all FPA IDs unique?

In [ ]:
query = '''
SELECT count(FPA_ID), count(distinct FPA_ID)
FROM Fires
WHERE (STATE='AK' OR STATE='WA' OR STATE='OR' OR STATE='CA');
'''
cursor.execute(query)
fpa_count_distinct = cursor.fetchall()
fpa_count_distinct

Very nearly all (except 1) FPA IDs are unique for West Coast states.

## Bar plot of COUNT(Fires) in West Coast States

In [ ]:
west_coast_fires_df = pd.DataFrame(west_coast_fires, columns=['State','Number of Fires'])

In [ ]:
west_coast_fires_df.plot(kind='bar', 
                        x='State')

## Filter for relevant states (with relevant columns) AND CREATE DATAFRAME

In [ ]:
columns_df_abbrev_west = columns_df[['FPA_ID', 
       'FIRE_YEAR', 'DISCOVERY_DOY',
       'STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR',
       'CONT_DOY', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE',
       'LONGITUDE', 'STATE']].copy()
columns_df_abbrev_west.head(0)

In [8]:
query = '''
SELECT FPA_ID, 
       FIRE_YEAR, DISCOVERY_DOY,
       STAT_CAUSE_CODE, STAT_CAUSE_DESCR,
       CONT_DOY, FIRE_SIZE, FIRE_SIZE_CLASS, LATITUDE,
       LONGITUDE, STATE
FROM Fires
WHERE (STATE='AK' OR STATE='WA' OR STATE='OR' OR STATE='CA');
'''
cursor.execute(query)
west = cursor.fetchall()
west_df = pd.DataFrame(west, columns=['fire_id','year','discovery_doy',
                                      'cause_code','cause_descr','contained_doy',
                                      'fire_size','fire_size_class',
                                      'lat','long','state'])

In [9]:
west_df.head()

,fire_id,year,discovery_doy,cause_code,cause_descr,contained_doy,fire_size,fire_size_class,lat,long,state
0,FS-1418826,2005,33,9.0,Miscellaneous,33.0,0.10,A,40.036944,-121.005833,CA
1,FS-1418827,2004,133,1.0,Lightning,133.0,0.25,A,38.933056,-120.404444,CA
2,FS-1418835,2004,152,5.0,Debris Burning,152.0,0.10,A,38.984167,-120.735556,CA
3,FS-1418845,2004,180,1.0,Lightning,185.0,0.10,A,38.559167,-119.913333,CA
4,FS-1418847,2004,180,1.0,Lightning,185.0,0.10,A,38.559167,-119.933056,CA


In [ ]:
west_df.info()

I seem to be missing some contained_doy values. (about 1/3 of discovery_doy values)

In [ ]:
west_df.describe()

In [ ]:
west_df.contained_doy

# Make sure STAT_CAUSE_DESCR = STAT_CAUSE_CODE

In [10]:
query = '''
select STAT_CAUSE_CODE, count(STAT_CAUSE_CODE), STAT_CAUSE_DESCR, count(STAT_CAUSE_DESCR) 
from Fires 
group by STAT_CAUSE_CODE;
'''
cursor.execute(query)
cause_codes = cursor.fetchall()
cause_codes_df = pd.DataFrame(cause_codes, columns=['Code','Code Count','Description','Descr Count'])
cause_codes_df

,Code,Code Count,Description,Descr Count
0,1.0,278468,Lightning,278468
1,2.0,147612,Equipment Use,147612
2,3.0,52869,Smoking,52869
3,4.0,76139,Campfire,76139
4,5.0,429028,Debris Burning,429028
5,6.0,33455,Railroad,33455
6,7.0,281455,Arson,281455
7,8.0,61167,Children,61167
8,9.0,323805,Miscellaneous,323805
9,10.0,11500,Fireworks,11500


The cause codes each seem to match with a single cause description (the lengths are the same), which means there are probably no entry errors here.

I want to drop the fires that were caused by Miscellaneous (323805) or Missing/Undefined (166723), because they are not helpful for prediction. Is that an ok thing to do?

Possible metacategories: natural, human_industry, human_recreation
OR natural vs human

## Store Cause Code/Description values in a dict

In [11]:
cause_codes_dict = pd.Series(cause_codes_df.Description.values,index=cause_codes_df.Code).to_dict()

In [12]:
cause_codes_dict

{1.0: 'Lightning',
 2.0: 'Equipment Use',
 3.0: 'Smoking',
 4.0: 'Campfire',
 5.0: 'Debris Burning',
 6.0: 'Railroad',
 7.0: 'Arson',
 8.0: 'Children',
 9.0: 'Miscellaneous',
 10.0: 'Fireworks',
 11.0: 'Powerline',
 12.0: 'Structure',
 13.0: 'Missing/Undefined'}

# FIRE DURATION (Days)

I expect Fire Size to be highly correlated with Fire Duration i.e. CONT_DATE - DISCOVERY_DATE. Is this the case?

In [ ]:
# figure this out later (or never)

# with FIRE_DURATION as (


# )

# What is the most common cause?

In [ ]:
query = '''
SELECT STAT_CAUSE_DESCR, COUNT(STAT_CAUSE_DESCR) 
FROM Fires 
GROUP BY STAT_CAUSE_DESCR 
ORDER BY COUNT(STAT_CAUSE_DESCR) DESC;
'''
cursor.execute(query)
causes = cursor.fetchall() # yields list of tuples in "causes" variable

In [ ]:
causes_df = pd.DataFrame(causes, columns=['Cause','Frequency'])

In [ ]:
causes_df.plot(kind='bar', x='Cause')

Most common cause across all years and locations: Debris Burning

## What happens if I remove Miscellaneous and Missing/Undefined?

In [ ]:
query = '''
SELECT STAT_CAUSE_DESCR, COUNT(STAT_CAUSE_DESCR) 
FROM Fires 
WHERE NOT (STAT_CAUSE_DESCR='Miscellaneous' OR STAT_CAUSE_DESCR='Missing/Undefined')
GROUP BY STAT_CAUSE_DESCR 
ORDER BY COUNT(STAT_CAUSE_DESCR) DESC;
'''
cursor.execute(query)
causes_pred = cursor.fetchall() 
causes_pred_df = pd.DataFrame(causes_pred, columns=['Cause','Frequency'])

In [ ]:
causes_pred_df.plot(kind='bar', x='Cause')

# How many fires in each year? (all locations)

In [ ]:
query = '''
SELECT FIRE_YEAR, COUNT(FOD_ID) 
FROM Fires 
GROUP BY FIRE_YEAR;
'''
cursor.execute(query)
fires_per_year = cursor.fetchall()
fires_per_year_df = pd.DataFrame(fires_per_year, columns=['Year','Number of Fires'])

In [ ]:
fires_per_year_df.plot(kind='bar', x='Year')

For some reason there were a lot of fires in 2007. It seems to be vaguely periodic, with peaks every 5-6 years.

# How many fires in each state? (all years)

In [ ]:
# cell takes ~50s to run
query = '''
SELECT STATE, COUNT(FOD_ID) 
FROM Fires 
GROUP BY STATE
ORDER BY COUNT(FOD_ID) DESC;
'''
cursor.execute(query)
fires_per_state = cursor.fetchall()
fires_per_state_df = pd.DataFrame(fires_per_state, columns=['State','Number of Fires'])

In [ ]:
fires_per_state_df.plot(kind='bar', 
                        x='State',
                       figsize=(12,4))


How does this relate to area/pop density of each state? (GA is not bigger than TX)

# Do fireworks cause more fires near the 4th of July? (DOY=185 or 186)

In [13]:
# STAT_CAUSE_CODE = 10.0 'Fireworks'
query = '''
SELECT DISCOVERY_DOY, count(DISCOVERY_DOY)
FROM Fires
where STAT_CAUSE_CODE=10
GROUP BY DISCOVERY_DOY;
'''
cursor.execute(query)
fireworks = cursor.fetchall()
fireworks_df = pd.DataFrame(fireworks, columns=['Discovery DOY', 'COUNT'])

In [ ]:
fireworks_df['COUNT'].sum()

In [ ]:
fireworks_df.plot(kind='bar', 
                  x='Discovery DOY',
                  figsize=(12,4))


Yields dramatic peak around July. Is this because of "fire season"?

Look at x range from May to September.

In [ ]:
fireworks_df_summer = fireworks_df.loc[160:210,]
fireworks_df_summer.plot(kind='bar', 
                  x='Discovery DOY',
                  figsize=(12,4),
                 xlim=(121,273)) # view DOY from May 1st to September 30th

# Pickle some stuff to use in a new notebook

In [14]:
import pickle

In [ ]:
# stuff_to_pickle = [columns_df_abbrev_west, west_df, cause_codes_dict, fireworks_df]

In [15]:
with open('columns_df_abbrev_west.pickle','wb') as to_write:
    pickle.dump(columns_df_abbrev_west, to_write)

In [16]:
with open('west_df.pickle','wb') as to_write:
    pickle.dump(west_df, to_write)

In [17]:
with open('cause_codes_dict.pickle','wb') as to_write:
    pickle.dump(cause_codes_dict, to_write)

In [18]:
with open('fireworks_df.pickle','wb') as to_write:
    pickle.dump(fireworks_df, to_write)

In [ ]:
# with open('my_dataframe.pickle', 'wb') as to_write:
#     pickle.dump(df, to_write)

# Appendix

Useful code for later

In [ ]:
sns.scatterplot('column1', 'column2', data=df, hue='label',
           palette=pal, alpha=.2, edgecolor=None)
plt.gcf().set_size_inches(10,6);